In [2]:
# Ejercicio 1

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

cursor = db.proveedores.find(
    {"activo": True, "habilitado": True},
    {"_id": 0, "CUIT": 1,"razon_social": 1, "tipo_sociedad": 1, "direccion": 1, "activo": 1, "habilitado": 1,"telefonos": 1}
)

with open("ejercicio1.txt", "w", encoding="utf-8") as file:
    for proveedor in cursor:
        file.write(str(proveedor) + "\n")

client.close()

In [3]:
# Ejercicio 4

from neo4j import GraphDatabase

uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

def get_proveedores_with_orders(tx):
    query = """
    MATCH (p:Proveedor)<-[:ES_DE]-(pedido:Pedido)
    RETURN DISTINCT p;
    """
    result = tx.run(query)
    return [record["p"] for record in result]

with driver.session() as session:
    proveedores = session.execute_read(get_proveedores_with_orders)

    with open("ejercicio4.txt", "w", encoding="utf-8") as file:
        for proveedor in proveedores:
            file.write(f"{proveedor}\n")

driver.close()


In [4]:
# Ejercicio 7

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

cursor = db.pedidos.aggregate([
    {
        "$lookup": {
        "from": "proveedores",
        "localField": "id_proveedor",
        "foreignField": "_id",
        "as": "proveedor"
        }
    },
    { "$unwind": "$proveedor" },
    {
        "$match": {
            "proveedor.CUIT": 30660608175
        }
    },
    {
        "$project": {
            "_id": 0,
            "id_proveedor": 1,
            "fecha": 1,
            "total_sin_iva": 1,
            "iva": 1,
            "detalle": 1
        }
    }
])

with open("ejercicio7.txt", "w", encoding="utf-8") as file:
    for pedido in cursor:
        file.write(str(pedido) + "\n")

client.close()


In [ ]:
# Ejercicio 10 

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

db.create_collection("ordenes_pedido_vista", viewOn="pedidos", pipeline=[
    {
        "$lookup": {
            "from": "proveedores",
            "localField": "id_proveedor",
            "foreignField": "_id",
            "as": "proveedor"
        }
    },
    { "$unwind": "$proveedor" },
    {
        "$project": {
            "_id": 0,
            "razon_social": "$proveedor.razon_social",
            "fecha": 1,
            "iva": 1,
            "total_sin_iva": 1,
            "total_con_iva": {
                "$add": ["$total_sin_iva", "$iva"]
            },
            "detalle": 1
        }
    },
    {
        "$sort": {
            "fecha": 1
        }
    }
])

# Consultar la vista
cursor = db.ordenes_pedido_vista.find()

# Escribir el resultado en el archivo (sobrescribiendo si ya existe)
with open("ejercicio10.txt", "w", encoding="utf-8") as file:
    for doc in cursor:
        file.write(str(doc) + "\n")

client.close()



In [ ]:
# Ejercicio 13 

from pymongo import MongoClient, errors as mongo_errors
from neo4j import GraphDatabase, exceptions as neo4j_errors

# Conexión a Mongo
client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

# Conexión a Neo4j
uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

# Función genérica para crear proveedor
def create_proveedor(id_proveedor, CUIT, razon_social, tipo_sociedad, direccion, activo, habilitado, telefonos):
    es_valido, mensaje = validar_telefonos(telefonos)
    if not es_valido:
        raise ValueError(f"Error en formato de teléfonos: {mensaje}")
    
    proveedor = {
        "_id": id_proveedor,
        "CUIT": CUIT,
        "razon_social": razon_social,
        "tipo_sociedad": tipo_sociedad,
        "direccion": direccion,
        "activo": activo,
        "habilitado": habilitado,
        "telefonos": telefonos
    }
    
    try:
        db.proveedores.insert_one(proveedor)
    except mongo_errors.PyMongoError as e:
        raise RuntimeError(f"Error al insertar en MongoDB: {e}")
    
    try:
        with driver.session() as session:
            def neo4j_tx(tx):
                tx.run("""
                    CREATE (p:Proveedor {
                        id_proveedor: $id_proveedor,
                        CUIT: $CUIT,
                        razon_social: $razon_social,
                        tipo_sociedad: $tipo_sociedad,
                        direccion: $direccion,
                        activo: $activo,
                        habilitado: $habilitado
                    })
                """,
                id_proveedor=id_proveedor,
                CUIT=CUIT,
                razon_social=razon_social,
                tipo_sociedad=tipo_sociedad,
                direccion=direccion,
                activo=activo,
                habilitado=habilitado
                )

                for tel in telefonos:
                    tx.run("""
                        MERGE (t:Telefono {
                            codigo_area: $codigo_area,
                            nro_telefono: $nro_telefono
                        })
                        SET t.tipo = $tipo
                        WITH t
                        MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                        MERGE (p)-[:TIENE_TELEFONO]->(t)
                    """, codigo_area=tel["codigo_area"],
                        nro_telefono=tel["nro_telefono"],
                        tipo=tel["tipo"],
                        id_proveedor=id_proveedor)
            
            session.execute_write(neo4j_tx)        
        
    except neo4j_errors.Neo4jError as e:
        db.proveedores.delete_one({"_id": id_proveedor})
        raise RuntimeError(f"Error al insertar en Neo4j, se revirtió la inserción en MongoDB: {e}")    

# Función para eliminar proveedor
def delete_proveedor(id_proveedor):
    proveedor_actual = db.proveedores.find_one({"_id": id_proveedor})
    if not proveedor_actual:
        raise ValueError("Proveedor no encontrado.")
    
    try:
        db.proveedores.delete_one({"_id": id_proveedor})
    except mongo_errors.PyMongoError as e:
        raise RuntimeError(f"Error al eliminar en MongoDB: {e}")
    
    try:
        with driver.session() as session:
            def neo4j_tx(tx):
                tx.run("""
                    MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                    DETACH DELETE p
                """, id_proveedor=id_proveedor)

            session.execute_write(neo4j_tx)

    except neo4j_errors.Neo4jError as e:
        try:
            db.proveedores.insert_one(proveedor_actual)
        except mongo_errors.PyMongoError as re:
            raise RuntimeError(f"Error al revertir MongoDB tras fallo en Neo4j: {re}") from e
        raise RuntimeError(f"Error al eliminar en Neo4j, se revirtió la eliminación en MongoDB: {e}")


# Función para modificar proveedor
def update_proveedor(id_proveedor, CUIT=None, razon_social=None, tipo_sociedad=None, direccion=None, activo=None, habilitado=None, telefonos=None):
    es_valido, mensaje = validar_telefonos(telefonos)
    if not es_valido:
        raise ValueError(f"Error en formato de teléfonos: {mensaje}")

    proveedor_actual = db.proveedores.find_one({"_id": id_proveedor})
    if not proveedor:
        raise ValueError("Proveedor no encontrado.")    
    
    updates = {}
    if CUIT is not None:
        updates["CUIT"] = CUIT
    if razon_social is not None:
        updates["razon_social"] = razon_social
    if tipo_sociedad is not None:
        updates["tipo_sociedad"] = tipo_sociedad
    if direccion is not None:
        updates["direccion"] = direccion
    if activo is not None:
        updates["activo"] = activo
    if habilitado is not None:
        updates["habilitado"] = habilitado
    if telefonos is not None:
        es_valido, mensaje = validar_telefonos(telefonos)
        if not es_valido:
            raise ValueError(f"Error en formato de teléfonos: {mensaje}")
        updates["telefonos"] = telefonos
    if not updates:
        raise ValueError("No se proporcionaron campos para actualizar.")

    try:
        db.proveedores.update_one({"_id": id_proveedor}, {"$set": updates})
    except mongo_errors.PyMongoError as e:
        raise RuntimeError(f"Error al actualizar en MongoDB: {e}")    

    try:
        with driver.session() as session:
            def neo4j_tx(tx):
                tx.run("""
                    MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                    SET p.CUIT = COALESCE($CUIT, p.CUIT),
                        p.razon_social = COALESCE($razon_social, p.razon_social),
                        p.tipo_sociedad = COALESCE($tipo_sociedad, p.tipo_sociedad),
                        p.direccion = COALESCE($direccion, p.direccion),
                        p.activo = COALESCE($activo, p.activo),
                        p.habilitado = COALESCE($habilitado, p.habilitado)
                """, id_proveedor=id_proveedor,
                     CUIT=CUIT,
                     razon_social=razon_social,
                     tipo_sociedad=tipo_sociedad,
                     direccion=direccion,
                     activo=activo,
                     habilitado=habilitado)

                if telefonos is not None:
                    tx.run("""
                        MATCH (p:Proveedor {id_proveedor: $id_proveedor})-[r:TIENE_TELEFONO]->(t:Telefono)
                        DELETE r
                    """, id_proveedor=id_proveedor)

                    for tel in telefonos:
                        tx.run("""
                            MERGE (t:Telefono {
                                codigo_area: $codigo_area,
                                nro_telefono: $nro_telefono
                            })
                            SET t.tipo = $tipo
                            WITH t
                            MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                            MERGE (p)-[:TIENE_TELEFONO]->(t)
                        """, codigo_area=tel["codigo_area"],
                             nro_telefono=tel["nro_telefono"],
                             tipo=tel["tipo"],
                             id_proveedor=id_proveedor)

            session.execute_write(neo4j_tx)

    except neo4j_errors.Neo4jError as e:
        try:
            db.proveedores.replace_one({"_id": id_proveedor}, proveedor_actual)
        except mongo_errors.PyMongoError as re:
            raise RuntimeError(f"Error al revertir MongoDB tras fallo en Neo4j: {re}") from e
        raise RuntimeError(f"Error al actualizar en Neo4j, se revirtió la actualización en MongoDB: {e}")
                

# Función para validar la lista de teléfonos
# Retorna True si es válida, False y un mensaje de error si no lo es.
def validar_telefonos(telefonos):
    if not isinstance(telefonos, list):
        return False, "La lista de teléfonos no es válida."

    numeros_vistos = set()

    for t in telefonos:
        if not isinstance(t, dict):
            return False, f"Teléfono inválido (no es un diccionario): {t}"

        if "codigo_area" not in t or not isinstance(t["codigo_area"], int):
            return False, f"Código de área inválido: {t}"

        if "nro_telefono" not in t or not isinstance(t["nro_telefono"], str):
            return False, f"Número de teléfono inválido: {t}"

        if "tipo" not in t or not isinstance(t["tipo"], str):
            return False, f"Tipo de teléfono inválido: {t}"

        clave = (t["codigo_area"], t["nro_telefono"])
        if clave in numeros_vistos:
            return False, f"Teléfono duplicado: {clave}"
        numeros_vistos.add(clave)

    return True, "OK"        

# Cerrar conexiones
client.close()
driver.close()

In [8]:
#   Ejercicio 3

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db = client["mi_base"]
proveedores = db["proveedores"]

query3 = proveedores.aggregate([
    {"$unwind": "$telefonos"},
    {"$project": {
        "_id": 0,
        "razon_social": 1,
        "CUIT": 1,
        "telefono": "$telefonos"
    }}
])
save_to_txt("query3.txt", list(query3))

In [9]:
#   Ejercicio 6

from pymongo import MongoClient
import json


client = MongoClient("mongodb://mongo:27017")

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db = client["mi_base"]
proveedores = db["proveedores"]

query6 = proveedores.aggregate([
    {"$lookup": {
        "from": "pedidos",
        "localField": "_id",
        "foreignField": "id_proveedor",
        "as": "ordenes"
    }},
    {"$project": {
        "_id": 0,
        "id_proveedor": "$_id",
        "razon_social": 1,
        "total_ordenes": {"$size": "$ordenes"},
        "monto_total_sin_iva": {
            "$sum": {"$map": {
                "input": "$ordenes",
                "as": "orden",
                "in": "$$orden.total_sin_iva"
            }}
        },
        "monto_total_con_iva": {
            "$sum": {
                "$map": {
                    "input": "$ordenes",
                    "as": "orden",
                    "in": {
                        "$add": ["$$orden.total_sin_iva", "$$orden.iva"]
                    }
                }
            }
        }
    }}
])
save_to_txt("query6.txt", list(query6))

In [10]:
#   Ejercicio 9

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]
pedidos = db["pedidos"]

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

query9 = pedidos.aggregate([
    {"$unwind": "$detalle"},
    {"$lookup": {
        "from": "productos",
        "localField": "detalle.id_producto",
        "foreignField": "_id",
        "as": "producto"
    }},
    {"$unwind": "$producto"},
    {"$match": {
        "producto.marca": "COTO"
    }},
    {"$project": {
        "_id": 1,
        "fecha": 1,
        "id_proveedor": 1,
        "total_sin_iva": 1,
        "iva": 1,
        "detalle": 1,
        "producto": 1
    }}
])
save_to_txt("query9.txt", list(query9))


In [11]:
#   Ejercicio 12

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db.command({
    "create": "proveedores_inhabilitados_activos",
    "viewOn": "proveedores",
    "pipeline": [
        {"$match": {
            "activo": True,
            "habilitado": False
        }},
        {"$project": {
            "_id": 0,
            "id_proveedor": "$_id",
            "razon_social": 1,
            "CUIT": 1,
            "direccion": 1,
            "tipo_sociedad": 1
        }}
    ]
})


## info de la vista en query12.txt
vista = db["proveedores_inhabilitados_activos"]
rta = list(vista.find())
save_to_txt("query12.txt", rta)


In [15]:
#   Ejercicio 15

from pymongo import MongoClient
from datetime import datetime
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]
proveedores = db["proveedores"]

## data de la orden 
nueva_orden = {
    "_id": 364,
    "id_proveedor": 1,
    "fecha": datetime.strptime("2025-06-19", "%Y-%m-%d"),
    "total_sin_iva": 1000.0,
    "iva": 210.0,
    "detalle": [
        {"id_producto": 1, "nro_item": 1, "cantidad": 3}
    ]
}

if proveedores.find_one({"_id": nueva_orden["id_proveedor"]}):
    pedidos.insert_one(nueva_orden)
    with open("query15.txt", "w") as f:
        f.write("Nueva orden registrada:\n")
        f.write(json.dumps(nueva_orden, default=str, indent=2))
else:
    with open("query15.txt", "w") as f:
        f.write(f"No se pudo registrar orden: proveedor con ID {nueva_orden['id_proveedor']} no existe.\n")

DuplicateKeyError: E11000 duplicate key error collection: mi_base.pedidos index: _id_ dup key: { _id: 364 }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: mi_base.pedidos index: _id_ dup key: { _id: 364 }', 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 364}}